# Preparation

In [128]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandasql as ps 
# this choice is based on preliminary analysis since it contained <100k data
# however, if its executed on google colab, it would be better to import and use spark instead
# yet it seems to be that the data uploaded to the cloud always get deleted once I log off from the lab
# This time I have to try my best not to blow my internal storage...
import warnings

pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [2]:
os.getcwd()

'C:\\Users\\Raymo'

In [3]:
origin_dir = os.getcwd()

In [4]:
# os.listdir()

In [5]:
# please download the package from https://physionet.org/content/mimiciii/1.4/ after getting access
# and extract the dowloaded file before executing this code
# this code should execute once only
os.chdir(origin_dir + '\mimic-iii-clinical-database-1.4\mimic-iii-clinical-database-1.4')

In [6]:
# os.getcwd()

In [7]:
os.listdir()

['ADMISSIONS.csv.gz',
 'CALLOUT.csv.gz',
 'CAREGIVERS.csv.gz',
 'CHARTEVENTS.csv.gz',
 'checksum_md5_unzipped.txt',
 'checksum_md5_zipped.txt',
 'CPTEVENTS.csv.gz',
 'DATETIMEEVENTS.csv.gz',
 'DIAGNOSES_ICD.csv.gz',
 'DRGCODES.csv.gz',
 'D_CPT.csv.gz',
 'D_ICD_DIAGNOSES.csv.gz',
 'D_ICD_PROCEDURES.csv.gz',
 'D_ITEMS.csv.gz',
 'D_LABITEMS.csv.gz',
 'ICUSTAYS.csv.gz',
 'INPUTEVENTS_CV.csv.gz',
 'INPUTEVENTS_MV.csv.gz',
 'LABEVENTS.csv.gz',
 'LICENSE.txt',
 'MICROBIOLOGYEVENTS.csv.gz',
 'NOTEEVENTS.csv.gz',
 'OUTPUTEVENTS.csv.gz',
 'PATIENTS.csv.gz',
 'PRESCRIPTIONS.csv.gz',
 'PROCEDUREEVENTS_MV.csv.gz',
 'PROCEDURES_ICD.csv.gz',
 'README.md',
 'SERVICES.csv.gz',
 'SHA256SUMS.txt',
 'TRANSFERS.csv.gz']

In [12]:
lst = os.listdir()
print(lst)

['ADMISSIONS.csv.gz', 'CALLOUT.csv.gz', 'CAREGIVERS.csv.gz', 'CHARTEVENTS.csv.gz', 'checksum_md5_unzipped.txt', 'checksum_md5_zipped.txt', 'CPTEVENTS.csv.gz', 'DATETIMEEVENTS.csv.gz', 'DIAGNOSES_ICD.csv.gz', 'DRGCODES.csv.gz', 'D_CPT.csv.gz', 'D_ICD_DIAGNOSES.csv.gz', 'D_ICD_PROCEDURES.csv.gz', 'D_ITEMS.csv.gz', 'D_LABITEMS.csv.gz', 'ICUSTAYS.csv.gz', 'INPUTEVENTS_CV.csv.gz', 'INPUTEVENTS_MV.csv.gz', 'LABEVENTS.csv.gz', 'LICENSE.txt', 'MICROBIOLOGYEVENTS.csv.gz', 'NOTEEVENTS.csv.gz', 'OUTPUTEVENTS.csv.gz', 'PATIENTS.csv.gz', 'PRESCRIPTIONS.csv.gz', 'PROCEDUREEVENTS_MV.csv.gz', 'PROCEDURES_ICD.csv.gz', 'README.md', 'SERVICES.csv.gz', 'SHA256SUMS.txt', 'TRANSFERS.csv.gz']


['ADMISSIONS.csv.gz',
 'CALLOUT.csv.gz',
 'CAREGIVERS.csv.gz',
 'CHARTEVENTS.csv.gz',
 'checksum_md5_unzipped.txt',
 'checksum_md5_zipped.txt',
 'CPTEVENTS.csv.gz',
 'DATETIMEEVENTS.csv.gz',
 'DIAGNOSES_ICD.csv.gz',
 'DRGCODES.csv.gz',
 'D_CPT.csv.gz',
 'D_ICD_DIAGNOSES.csv.gz',
 'D_ICD_PROCEDURES.csv.gz',
 'D_ITEMS.csv.gz',
 'D_LABITEMS.csv.gz',
 'ICUSTAYS.csv.gz',
 'INPUTEVENTS_CV.csv.gz',
 'INPUTEVENTS_MV.csv.gz',
 'LABEVENTS.csv.gz',
 'LICENSE.txt',
 'MICROBIOLOGYEVENTS.csv.gz',
 'NOTEEVENTS.csv.gz',
 'OUTPUTEVENTS.csv.gz',
 'PATIENTS.csv.gz',
 'PRESCRIPTIONS.csv.gz',
 'PROCEDUREEVENTS_MV.csv.gz',
 'PROCEDURES_ICD.csv.gz',
 'README.md',
 'SERVICES.csv.gz',
 'SHA256SUMS.txt',
 'TRANSFERS.csv.gz']<br>
 There are plenty of csv files stored as `.gz` file. Let's explore what are inside and select <font color = 'red'>useful </font>datafiles inside. 

# File Exploration
For patient triage, before an in-depth exploration, I would like to browse each data file to see which feature is useful, and more importantly, how to join different tables <b>correctly</b>.

### 1. `ADMISSIONS.csv.gz`

In [110]:
admissions = pd.read_csv('ADMISSIONS.csv.gz', compression='gzip', error_bad_lines=False)
# I could use lst[1] as file name instead, however, if the file is relocated to the different directory or removed,
# it would be more buggy...
admissions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ADMITTIME,DISCHTIME,DEATHTIME,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,LANGUAGE,RELIGION,MARITAL_STATUS,ETHNICITY,EDREGTIME,EDOUTTIME,DIAGNOSIS,HOSPITAL_EXPIRE_FLAG,HAS_CHARTEVENTS_DATA
0,21,22,165315,2196-04-09 12:26:00,2196-04-10 15:54:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,DISC-TRAN CANCER/CHLDRN H,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2196-04-09 10:06:00,2196-04-09 13:24:00,BENZODIAZEPINE OVERDOSE,0,1
1,22,23,152223,2153-09-03 07:15:00,2153-09-08 19:10:00,NaN,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,MARRIED,WHITE,NaN,NaN,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,0,1
2,23,23,124321,2157-10-18 19:34:00,2157-10-25 14:00:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME HEALTH CARE,Medicare,ENGL,CATHOLIC,MARRIED,WHITE,NaN,NaN,BRAIN MASS,0,1
3,24,24,161859,2139-06-06 16:14:00,2139-06-09 12:48:00,NaN,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,HOME,Private,NaN,PROTESTANT QUAKER,SINGLE,WHITE,NaN,NaN,INTERIOR MYOCARDIAL INFARCTION,0,1
4,25,25,129635,2160-11-02 02:06:00,2160-11-05 14:55:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME,Private,NaN,UNOBTAINABLE,MARRIED,WHITE,2160-11-02 01:01:00,2160-11-02 04:27:00,ACUTE CORONARY SYNDROME,0,1


In [111]:
admissions.shape

(58976, 19)

* For `ADMISSION`, I'd like to select `ADMISSION_TYPE` = "EMERGENCY" in SQL, before joining other data on `HADM_ID`.<br>
* For this table, apart from `SUBJECT_ID`, `HADM_ID`, I need `ADMITTIME`, `ETHNICITY` (basic demographic!!!) for the dataset.
* ps: the time stamp tends to be strange (despite considerating other calendars in different cultures). But whatever...

### 2. `CALLOUT.csv.gz`

In [112]:
callout = pd.read_csv('CALLOUT.csv.gz', compression = 'gzip')
callout.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SUBMIT_WARDID,SUBMIT_CAREUNIT,CURR_WARDID,CURR_CAREUNIT,CALLOUT_WARDID,CALLOUT_SERVICE,REQUEST_TELE,REQUEST_RESP,REQUEST_CDIFF,REQUEST_MRSA,REQUEST_VRE,CALLOUT_STATUS,CALLOUT_OUTCOME,DISCHARGE_WARDID,ACKNOWLEDGE_STATUS,CREATETIME,UPDATETIME,ACKNOWLEDGETIME,OUTCOMETIME,FIRSTRESERVATIONTIME,CURRENTRESERVATIONTIME
0,402,854,175684,52.0,NaN,29.0,MICU,1,MED,0,0,0,0,0,Inactive,Discharged,29.0,Acknowledged,2146-10-05 13:16:55,2146-10-05 13:16:55,2146-10-05 13:24:00,2146-10-05 18:55:22,2146-10-05 15:27:44,NaN
1,403,864,138624,15.0,NaN,55.0,CSRU,55,CSURG,0,0,0,0,0,Inactive,Discharged,55.0,Acknowledged,2114-11-28 08:31:39,2114-11-28 09:42:08,2114-11-28 09:43:08,2114-11-28 12:10:02,NaN,NaN
2,404,864,138624,12.0,NaN,55.0,CSRU,55,CSURG,1,0,0,0,0,Inactive,Discharged,55.0,Acknowledged,2114-11-30 10:24:25,2114-12-01 09:06:18,2114-12-01 12:26:05,2114-12-01 21:55:05,NaN,NaN
3,405,867,184298,7.0,NaN,17.0,CCU,17,CCU,1,1,0,0,0,Inactive,Discharged,17.0,Acknowledged,2136-12-29 08:45:42,2136-12-29 10:17:16,2136-12-29 10:33:51,2136-12-29 18:10:02,NaN,NaN
4,157,306,167129,57.0,NaN,3.0,SICU,44,NSURG,1,0,0,0,0,Inactive,Discharged,3.0,Acknowledged,2199-09-18 11:47:47,2199-09-18 11:47:47,2199-09-18 11:58:33,2199-09-18 15:10:02,NaN,NaN


In [113]:
callout.shape

(34499, 24)

* For `CALLOUT`, it is relevant to the log record for the call out activity.
* Nothing interesting for this project.

### 3. `CAREGIVERS.csv.gz`

In [132]:
caregivers = pd.read_csv('CAREGIVERS.csv.gz', compression = 'gzip')
caregivers.head()

,ROW_ID,CGID,LABEL,DESCRIPTION
0,2228,16174,RO,Read Only
1,2229,16175,RO,Read Only
2,2230,16176,Res,Resident/Fellow/PA/NP
3,2231,16177,RO,Read Only
4,2232,16178,RT,Respiratory


In [133]:
print(caregivers.shape)

(7567, 4)


* for `CAREGIVERS`, this is a namelist (ID) of each caregive.
* Nothing intersting for this project.

### 4. `CHARTEVENTS.csv.gz`

In [136]:
chart_events = pd.read_csv('CHARTEVENTS.csv.gz', compression = 'gzip')
chart_events.head()
# very large, requiring an excessive time to read...

KeyboardInterrupt: 

In [ ]:
chart_events.shape

Due to the excessive size, this file was accessed through excel.
* This file contains the exam result & record for each patient
* The result seems to be for the basic patient sign, very valuable
* need a massive <font color = 'red'><b>PIVOT</b></font> to reassign them as `Temperature`, `Breath`, `Pulse`, `Blood_Pressure`, and, if possible, count the # of alerting value.
* if such a table join with others, it would be excessively massive... therefore, `spark` would be needed to undergo such a project.

### 5. `checksum_md5_unzipped.txt`
For `.txt` file like this, I would like to have a look at its content. It might provide additional info for this data collection.

In [64]:
checksum_doc = open('checksum_md5_unzipped.txt', 'r')

In [65]:
print(checksum_doc.read())
checksum_doc.seek(0)
print(f'The document has {len(checksum_doc.readlines())} lines')

57d940b69dd066da5ba57e008cc7f92c  ADMISSIONS.csv
cd4e416337e68f6678ff4e091938be58  CALLOUT.csv
258e3c2bc11798c99ffc4f33aa1e9bcd  CAREGIVERS.csv
2806d07875905b239de21057c4e7010d  CHARTEVENTS.csv
bf32e07ecd2b946a675a0d7cea75d2e7  CPTEVENTS.csv
6245f2f4f581dce8e05e147228a51522  DATETIMEEVENTS.csv
49fdda583f5e85e5e0a92686b3106fac  D_CPT.csv
e2c0a05d768a6273038ae84e576186a7  DIAGNOSES_ICD.csv
d0a7026ca3618f4360d8329643d9041d  D_ICD_DIAGNOSES.csv
a9707e5361f939f45ca2bc2eb8bd5652  D_ICD_PROCEDURES.csv
749e350c22531ec8589b7391b4e7b660  D_ITEMS.csv
2f77db8fc2f2a21e4fad1a1781d98709  D_LABITEMS.csv
ea61c7dfe180c6d6a1273f220b5e70c5  DRGCODES.csv
b2a57affcda3c60fa38a022b2df7fcf2  ICUSTAYS.csv
5fce8501d6723a470c74affcda32e52b  INPUTEVENTS_CV.csv
4d57864670f51e7230c0fef52d206049  INPUTEVENTS_MV.csv
bc2fe94983576207758635924c047dbe  LABEVENTS.csv
0a35833252bdecb32b9d1348adec2085  MICROBIOLOGYEVENTS.csv
df33ab9764256b34bfc146828f440c2b  NOTEEVENTS.csv
6e137e00d7a7fd14c291d015573ed375  OUTPUTEVENTS.csv


It is a list of all the csv files within the dataset.

In [66]:
checksum_doc.close()

### 6. `checksum_md5_zipped.txt`

In [69]:
checksum_zip = open('checksum_md5_zipped.txt', 'r')
print(checksum_zip.read())
checksum_zip.seek(0)
print(f'This document has {len(checksum_zip.readlines())} rows.')
checksum_zip.close()

f301f427b38268ae9b3e129f49484b8d  ADMISSIONS.csv.gz
f9443602f929725d0d240edde9c37848  CALLOUT.csv.gz
14e103974ac30c7e8ec92fa15966730a  CAREGIVERS.csv.gz
31f4ecfd69b24ec88d58e270af51ab12  CHARTEVENTS.csv.gz
88ec8e5aa63f3f50c49b67482a0a691d  CPTEVENTS.csv.gz
68b7b786469c66af182bc0d41a4d9e69  DATETIMEEVENTS.csv.gz
4801e130461a147e647cd4c3e4579208  D_CPT.csv.gz
1d8007cc3115fd87a95321df33e1de86  DIAGNOSES_ICD.csv.gz
9c5b6a1e0c6ebe7a96b99441ca9c0499  D_ICD_DIAGNOSES.csv.gz
7927c2e262485b69ae6dd31359faa664  D_ICD_PROCEDURES.csv.gz
83413729fa45a4d594e9a530bfbad911  D_ITEMS.csv.gz
75e79ca14a2b8c02e3535b87498c4f9d  D_LABITEMS.csv.gz
34f5b2327cda3ffa53089b6c23b53191  DRGCODES.csv.gz
79f2b70cb07a6598642e04aaa364c27c  ICUSTAYS.csv.gz
1d02246e9fd8d379a4e0c41623890c25  INPUTEVENTS_CV.csv.gz
f49043f52cd3ee121f36b66e5504a9c0  INPUTEVENTS_MV.csv.gz
9d78e309dc5b5fcf80e34803a688a354  LABEVENTS.csv.gz
b600550677d8dd8d13184f40c5162093  MICROBIOLOGYEVENTS.csv.gz
90c05ebe5e7e631f0a55f34f568fbfc0  NOTEEVENTS.c

So this document contains the name list of 26 extracted csv files stored in `.gz`.

### 7. `CPTEVENTS.csv.gz`

In [138]:
cpt_events = pd.read_csv('CPTEVENTS.csv.gz', compression = 'gzip')
cpt_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,COSTCENTER,CHARTDATE,CPT_CD,CPT_NUMBER,CPT_SUFFIX,TICKET_ID_SEQ,SECTIONHEADER,SUBSECTIONHEADER,DESCRIPTION
0,317,11743,129545,ICU,NaN,99232,99232.0,NaN,6.0,Evaluation and management,Hospital inpatient services,NaN
1,318,11743,129545,ICU,NaN,99232,99232.0,NaN,7.0,Evaluation and management,Hospital inpatient services,NaN
2,319,11743,129545,ICU,NaN,99232,99232.0,NaN,8.0,Evaluation and management,Hospital inpatient services,NaN
3,320,11743,129545,ICU,NaN,99232,99232.0,NaN,9.0,Evaluation and management,Hospital inpatient services,NaN
4,321,6185,183725,ICU,NaN,99223,99223.0,NaN,1.0,Evaluation and management,Hospital inpatient services,NaN


In [139]:
cpt_events.shape

(573146, 12)

In [140]:
cpt_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 573146 entries, 0 to 573145
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   ROW_ID            573146 non-null  int64  
 1   SUBJECT_ID        573146 non-null  int64  
 2   HADM_ID           573146 non-null  int64  
 3   COSTCENTER        573146 non-null  object 
 4   CHARTDATE         101545 non-null  object 
 5   CPT_CD            573146 non-null  object 
 6   CPT_NUMBER        573128 non-null  float64
 7   CPT_SUFFIX        22 non-null      object 
 8   TICKET_ID_SEQ     471601 non-null  float64
 9   SECTIONHEADER     573125 non-null  object 
 10  SUBSECTIONHEADER  573125 non-null  object 
 11  DESCRIPTION       101545 non-null  object 
dtypes: float64(2), int64(3), object(7)
memory usage: 52.5+ MB


* `CPTEVENTS` stores the medical terminological coding (CPT) of patient healthcare record
* Nothing interesting for this project.

### 8. `DATETIMEEVENTS.csv.gz`

In [141]:
date_time_events = pd.read_csv('DATETIMEEVENTS.csv.gz', compression = 'gzip')
date_time_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,ITEMID,CHARTTIME,STORETIME,CGID,VALUE,VALUEUOM,WARNING,ERROR,RESULTSTATUS,STOPPED
0,711,7657,121183.0,297945.0,3411,2172-03-14 11:00:00,2172-03-14 11:52:00,16446,NaN,Date,NaN,NaN,NaN,NotStopd
1,712,7657,121183.0,297945.0,3411,2172-03-14 13:00:00,2172-03-14 12:36:00,16446,NaN,Date,NaN,NaN,NaN,NotStopd
2,713,7657,121183.0,297945.0,3411,2172-03-14 15:00:00,2172-03-14 15:10:00,14957,NaN,Date,NaN,NaN,NaN,NotStopd
3,714,7657,121183.0,297945.0,3411,2172-03-14 17:00:00,2172-03-14 17:01:00,16446,NaN,Date,NaN,NaN,NaN,NotStopd
4,715,7657,121183.0,297945.0,3411,2172-03-14 19:00:00,2172-03-14 19:29:00,14815,NaN,Date,NaN,NaN,NaN,NotStopd


In [142]:
date_time_events.shape

(4485937, 14)

In [144]:
date_time_events.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4485937 entries, 0 to 4485936
Data columns (total 14 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   ROW_ID        4485937 non-null  int64  
 1   SUBJECT_ID    4485937 non-null  int64  
 2   HADM_ID       4484165 non-null  float64
 3   ICUSTAY_ID    4481781 non-null  float64
 4   ITEMID        4485937 non-null  int64  
 5   CHARTTIME     4485937 non-null  object 
 6   STORETIME     4485937 non-null  object 
 7   CGID          4485937 non-null  int64  
 8   VALUE         4433847 non-null  object 
 9   VALUEUOM      4485937 non-null  object 
 10  WARNING       2686920 non-null  float64
 11  ERROR         2686920 non-null  float64
 12  RESULTSTATUS  0 non-null        float64
 13  STOPPED       1799017 non-null  object 
dtypes: float64(5), int64(4), object(5)
memory usage: 479.1+ MB


* `DATETIMEEVENTS` records the warning events of patient during hospital stay.
* This project is for home-to-emergency room patient triage. Therefore, nothing interesting for this.

### 9. `DIAGNOSES_ICD.csv.gz`

In [146]:
diagnosis_icd = pd.read_csv('DIAGNOSES_ICD.csv.gz', compression = 'gzip')
diagnosis_icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [147]:
diagnosis_icd.shape

(651047, 5)

* `DIAGNOSES_ICD` stores the ICD code (disease & problem) for each patient.
* It is relevant to the medical terminology. Therefore, nothing interesting here.

### 10. `DRGCODES.csv.gz`

In [148]:
drg_codes = pd.read_csv('DRGCODES.csv.gz', compression = 'gzip')
drg_codes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,DRG_TYPE,DRG_CODE,DESCRIPTION,DRG_SEVERITY,DRG_MORTALITY
0,342,2491,144486,HCFA,28,"TRAUMATIC STUPOR & COMA, COMA <1 HR AGE >17 WI...",NaN,NaN
1,343,24958,162910,HCFA,110,MAJOR CARDIOVASCULAR PROCEDURES WITH COMPLICAT...,NaN,NaN
2,344,18325,153751,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN
3,345,17887,182692,HCFA,14,SPECIFIC CEREBROVASCULAR DISORDERS EXCEPT TRAN...,NaN,NaN
4,346,11113,157980,HCFA,390,NEONATE WITH OTHER SIGNIFICANT PROBLEMS,NaN,NaN


In [150]:
drg_codes.shape

(125557, 8)

* for `DRGCODES`, it is mainly about the coding of patient problem by DRG terminology.
* Nothing interesting for this project.

### 11. `D_CPT.csv.gz`

In [151]:
d_cpt = pd.read_csv('D_CPT.csv.gz', compression = 'gzip')
d_cpt.head()

,ROW_ID,CATEGORY,SECTIONRANGE,SECTIONHEADER,SUBSECTIONRANGE,SUBSECTIONHEADER,CODESUFFIX,MINCODEINSUBSECTION,MAXCODEINSUBSECTION
0,1,1,99201-99499,Evaluation and management,99201-99216,Office/other outpatient services,NaN,99201,99216
1,2,1,99201-99499,Evaluation and management,99217-99220,Hospital observation services,NaN,99217,99220
2,3,1,99201-99499,Evaluation and management,99221-99239,Hospital inpatient services,NaN,99221,99239
3,4,1,99201-99499,Evaluation and management,99241-99255,Consultations,NaN,99241,99255
4,5,1,99201-99499,Evaluation and management,99261-99263,Follow-up inpatient consultations (deleted codes),NaN,99261,99263


In [152]:
d_cpt.shape

(134, 9)

* `D_CPT` is a spreadsheet about CPT encoding.
* Nothing interesting for my project.

### 12. `D_ICD_DIAGNOSES.csv.gz`

In [153]:
d_icd_diagnoses = pd.read_csv('D_ICD_DIAGNOSES.csv.gz', compression = 'gzip')
d_icd_diagnoses.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [154]:
d_icd_diagnoses.shape

(14567, 4)

* `D_ICD_DIAGNOSES` is a spreadsheet for the ICD9 code of the corresponding disease
* Nothing interesting for this project

### 13. `D_ICD_PROCEDURES.csv.gz`

In [155]:
d_icd_procedures = pd.read_csv('D_ICD_DIAGNOSES.csv.gz', compression = 'gzip')
d_icd_procedures.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [157]:
d_icd_procedures.shape

(14567, 4)

* `D_ICD_DIAGNOSES` is about the ICD9 coding for each medical intervention.
* Nothing interesting for this project, unless for seeking the detailed terminological information through joining.

### 14. `D_ITEMS.csv.gz`

In [158]:
d_items = pd.read_csv('D_ITEMS.csv.gz', compression = 'gzip')
d_items.head()

,ROW_ID,ITEMID,LABEL,ABBREVIATION,DBSOURCE,LINKSTO,CATEGORY,UNITNAME,PARAM_TYPE,CONCEPTID
0,457,497,Patient controlled analgesia (PCA) [Inject],NaN,carevue,chartevents,NaN,NaN,NaN,NaN
1,458,498,PCA Lockout (Min),NaN,carevue,chartevents,NaN,NaN,NaN,NaN
2,459,499,PCA Medication,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
3,460,500,PCA Total Dose,NaN,carevue,chartevents,NaN,NaN,NaN,NaN
4,461,501,PCV Exh Vt (Obser),NaN,carevue,chartevents,NaN,NaN,NaN,NaN


In [159]:
d_items.shape

(12487, 10)

* `D_ITEMS` contains the detailed information of medication.
* Nothing interesting for our project.

### 15. `D_LABITEMS.csv.gz`

In [160]:
d_lab_items = pd.read_csv('D_LABITEMS.csv.gz', compression = 'gzip')
d_lab_items.head()

,ROW_ID,ITEMID,LABEL,FLUID,CATEGORY,LOINC_CODE
0,546,51346,Blasts,Cerebrospinal Fluid (CSF),Hematology,26447-3
1,547,51347,Eosinophils,Cerebrospinal Fluid (CSF),Hematology,26451-5
2,548,51348,"Hematocrit, CSF",Cerebrospinal Fluid (CSF),Hematology,30398-2
3,549,51349,Hypersegmented Neutrophils,Cerebrospinal Fluid (CSF),Hematology,26506-6
4,550,51350,Immunophenotyping,Cerebrospinal Fluid (CSF),Hematology,NaN


In [161]:
d_lab_items.shape

(753, 6)

* `D_LABITEMS` contains detailed information of medical test
* Nothing interesting for this project.

### 16. `ICUSTAYS.csv.gz`

In [162]:
icu_stay = pd.read_csv('ICUSTAYS.csv.gz', compression = 'gzip')
icu_stay.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,FIRST_CAREUNIT,LAST_CAREUNIT,FIRST_WARDID,LAST_WARDID,INTIME,OUTTIME,LOS
0,365,268,110404,280836,carevue,MICU,MICU,52,52,2198-02-14 23:27:38,2198-02-18 05:26:11,3.2490
1,366,269,106296,206613,carevue,MICU,MICU,52,52,2170-11-05 11:05:29,2170-11-08 17:46:57,3.2788
2,367,270,188028,220345,carevue,CCU,CCU,57,57,2128-06-24 15:05:20,2128-06-27 12:32:29,2.8939
3,368,271,173727,249196,carevue,MICU,SICU,52,23,2120-08-07 23:12:42,2120-08-10 00:39:04,2.0600
4,369,272,164716,210407,carevue,CCU,CCU,57,57,2186-12-25 21:08:04,2186-12-27 12:01:13,1.6202


In [163]:
icu_stay.shape

(61532, 12)

* `ICUSTAYS` contains the `time`, `careunit` of the patient ICU stay, with `LOS` as its staying duration.
* Nothing interesting for this project. <font color = '#55BBFF'><b>Unless for predicting length of stay</b>, which would be a valuable datasource.</font>

### 17. `INPUTEVENTS_CV.csv.gz`

In [164]:
input_events_cv = pd.read_csv('INPUTEVENTS_CV.csv.gz', compression = 'gzip')
input_events_cv.head()
# again, it would take a bit long to open it

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,STORETIME,CGID,ORDERID,LINKORDERID,STOPPED,NEWBOTTLE,ORIGINALAMOUNT,ORIGINALAMOUNTUOM,ORIGINALROUTE,ORIGINALRATE,ORIGINALRATEUOM,ORIGINALSITE
0,592,24457,184834.0,205776.0,2193-09-11 09:00:00,30056,100.0,ml,NaN,NaN,2193-09-11 11:12:00,14990.0,756654,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
1,593,24457,184834.0,205776.0,2193-09-11 12:00:00,30056,200.0,ml,NaN,NaN,2193-09-11 12:05:00,14990.0,3564075,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
2,594,24457,184834.0,205776.0,2193-09-11 16:00:00,30056,160.0,ml,NaN,NaN,2193-09-11 16:52:00,14990.0,422646,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
3,595,24457,184834.0,205776.0,2193-09-11 19:00:00,30056,240.0,ml,NaN,NaN,2193-09-11 19:06:00,14990.0,5137889,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN
4,596,24457,184834.0,205776.0,2193-09-11 21:00:00,30056,50.0,ml,NaN,NaN,2193-09-11 21:19:00,15029.0,8343792,9359133,NaN,NaN,NaN,ml,Oral,NaN,NaN,NaN


In [165]:
input_events_cv.shape

(17527935, 22)

In [166]:
input_events_cv.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17527935 entries, 0 to 17527934
Data columns (total 22 columns):
 #   Column             Non-Null Count     Dtype  
---  ------             --------------     -----  
 0   ROW_ID             17527935 non-null  int64  
 1   SUBJECT_ID         17527935 non-null  int64  
 2   HADM_ID            17505266 non-null  float64
 3   ICUSTAY_ID         17497482 non-null  float64
 4   CHARTTIME          17527935 non-null  object 
 5   ITEMID             17527935 non-null  int64  
 6   AMOUNT             12422557 non-null  float64
 7   AMOUNTUOM          13195400 non-null  object 
 8   RATE               4132022 non-null   float64
 9   RATEUOM            4172256 non-null   object 
 10  STORETIME          17527935 non-null  object 
 11  CGID               17173928 non-null  float64
 12  ORDERID            17527935 non-null  int64  
 13  LINKORDERID        17527935 non-null  int64  
 14  STOPPED            480551 non-null    object 
 15  NEWBOTTLE    

* `INPUTEVENTS_CV` stores the medical intervention, such as drug treatment for each patient.
* `ITEM_ID` could be used to join the table with `D_ITEMS` or `D_LABITEMS`.
* Nothing interesting for this project.

### 18. `INPUTEVENTS_MV.csv.gz`

In [167]:
input_events_mv = pd.read_csv('INPUTEVENTS_MV.csv.gz', compression = 'gzip')
input_events_mv.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,AMOUNT,AMOUNTUOM,RATE,RATEUOM,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCOMPONENTTYPEDESCRIPTION,ORDERCATEGORYDESCRIPTION,PATIENTWEIGHT,TOTALAMOUNT,TOTALAMOUNTUOM,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE,ORIGINALAMOUNT,ORIGINALRATE
0,241,27063,139787,223259.0,2133-02-05 06:29:00,2133-02-05 08:45:00,225166,6.774532,mEq,NaN,NaN,2133-02-05 06:27:00,16009,4892074,4892074,02-Fluids (Crystalloids),Additive (Crystalloid),Additives ...,Continuous IV,83.2,100.0,ml,0,0,1,Rewritten,NaN,RN,2133-02-05 12:52:00,10.000000,0.050000
1,242,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225944,28.132997,ml,30.142497,mL/hour,2133-02-05 12:57:00,20053,5211428,1576890,02-Fluids (Crystalloids),Additive (Crystalloid),Main order parameter,Continuous IV,83.2,100.0,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,28.132998,30.255817
2,243,27063,139787,223259.0,2133-02-05 05:34:00,2133-02-05 06:30:00,225166,2.813300,mEq,NaN,NaN,2133-02-05 12:57:00,20053,5211428,1576890,02-Fluids (Crystalloids),Additive (Crystalloid),Additives ...,Continuous IV,83.2,100.0,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,2.813300,0.050426
3,244,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-03 12:01:00,225893,1.000000,dose,NaN,NaN,2133-02-03 13:21:00,20030,5213971,5213971,08-Antibiotics (IV),02-Fluids (Crystalloids),Main order parameter,Drug Push,83.2,100.0,ml,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,1.000000,1.000000
4,245,27063,139787,223259.0,2133-02-03 12:00:00,2133-02-03 12:01:00,220949,100.000000,ml,NaN,NaN,2133-02-03 13:21:00,20030,5213971,5213971,08-Antibiotics (IV),02-Fluids (Crystalloids),Mixed solution,Drug Push,83.2,100.0,ml,0,0,2,Rewritten,RN,NaN,2133-02-03 17:06:00,100.000000,0.000000


In [168]:
input_events_mv.shape

(3618991, 31)

In [170]:
input_events_mv.info(verbose = True, show_counts = True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3618991 entries, 0 to 3618990
Data columns (total 31 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   ROW_ID                         3618991 non-null  int64  
 1   SUBJECT_ID                     3618991 non-null  int64  
 2   HADM_ID                        3618991 non-null  int64  
 3   ICUSTAY_ID                     3617827 non-null  float64
 4   STARTTIME                      3618991 non-null  object 
 5   ENDTIME                        3618991 non-null  object 
 6   ITEMID                         3618991 non-null  int64  
 7   AMOUNT                         3618991 non-null  float64
 8   AMOUNTUOM                      3618991 non-null  object 
 9   RATE                           2042852 non-null  float64
 10  RATEUOM                        2042852 non-null  object 
 11  STORETIME                      3618991 non-null  object 
 12  CGID          

* for `INPUTEVENTS_MV`, it contains medical intervention record for each patient.
* Nothing interesting for this project.

### 19. `LABEVENTS.csv.gz`

In [171]:
lab_events = pd.read_csv('LABEVENTS.csv.gz', compression = 'gzip')
lab_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ITEMID,CHARTTIME,VALUE,VALUENUM,VALUEUOM,FLAG
0,281,3,NaN,50820,2101-10-12 16:07:00,7.39,7.39,units,NaN
1,282,3,NaN,50800,2101-10-12 18:17:00,ART,NaN,NaN,NaN
2,283,3,NaN,50802,2101-10-12 18:17:00,-1,-1.00,mEq/L,NaN
3,284,3,NaN,50804,2101-10-12 18:17:00,22,22.00,mEq/L,NaN
4,285,3,NaN,50808,2101-10-12 18:17:00,0.93,0.93,mmol/L,abnormal


In [172]:
lab_events.shape

(27854055, 9)

* For `LABEVENTS`, it contains lab test records within given Healthcare Institutes.
* Nothing interesting for this project

### 20. `LICENSE.txt`

In [90]:
license = open('LICENSE.txt', 'r')
print(license.read())
license.close()

The PhysioNet Credentialed Health Data License
Version 1.4.0

Copyright (c) 2019 MIT Laboratory for Computational Physiology

The MIT Laboratory for Computational Physiology (MIT-LCP) wishes 
to make data available for research and educational purposes to 
qualified requestors, but only if the data are used and protected 
in accordance with the terms and conditions stated in this License.

It is hereby agreed between the data requestor, hereinafter referred to
as the "LICENSEE", and MIT-LCP, that:

1. The LICENSEE will not attempt to identify any individual or
   institution referenced in PhysioNet restricted data.  
2. The LICENSEE will exercise all reasonable and prudent care to avoid
   disclosure of the identity of any individual or institution
   referenced in PhysioNet restricted data in any publication or other
   communication.  
3. The LICENSEE will not share access to PhysioNet restricted data
   with anyone else. 
4. The LICENSEE will exercise all reasonable and prudent care

This is the license of the dataset

### 21. `MICROBIOLOGYEVENTS.csv.gz`

In [174]:
microbiology_events = pd.read_csv('MICROBIOLOGYEVENTS.csv.gz', compression = 'gzip')
microbiology_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,SPEC_ITEMID,SPEC_TYPE_DESC,ORG_ITEMID,ORG_NAME,ISOLATE_NUM,AB_ITEMID,AB_NAME,DILUTION_TEXT,DILUTION_COMPARISON,DILUTION_VALUE,INTERPRETATION
0,744,96,170324,2156-04-13 00:00:00,2156-04-13 14:18:00,70021.0,BRONCHOALVEOLAR LAVAGE,80026.0,PSEUDOMONAS AERUGINOSA,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,745,96,170324,2156-04-20 00:00:00,2156-04-20 13:10:00,70062.0,SPUTUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,746,96,170324,2156-04-20 00:00:00,2156-04-20 16:00:00,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,747,96,170324,2156-04-20 00:00:00,NaN,70012.0,BLOOD CULTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,748,96,170324,2156-04-20 00:00:00,NaN,70079.0,URINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [175]:
microbiology_events.shape

(631726, 16)

* `MICROBIOLOGYEVENTS` is a spread sheet about the microbiological examen record in the corresponding institutes.
* Nothing interesting for this project.

### 22. `NOTEEVENTS.csv.gz`

In [177]:
note_events = pd.read_csv('NOTEEVENTS.csv.gz', compression = 'gzip')
note_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [178]:
note_events.shape

(2083180, 11)

In [187]:
note_events.groupby(['CATEGORY'])['CATEGORY'].count()

CATEGORY
Case Management         967
Consult                  98
Discharge summary     59652
ECG                  209051
Echo                  45794
General                8301
Nursing              223556
Nursing/other        822497
Nutrition              9418
Pharmacy                103
Physician            141624
Radiology            522279
Rehab Services         5431
Respiratory           31739
Social Work            2670
Name: CATEGORY, dtype: int64

* `NOTEEVENTS` is a spreadsheet about the clinical notes by each category.
* <font color = 'red'><b>The one with chief complain may be useful for the project</b></font>

### 23. `OUTPUTEVENTS.csv.gz`

In [188]:
output_events = pd.read_csv('OUTPUTEVENTS.csv.gz', compression = 'gzip')
output_events.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,CHARTTIME,ITEMID,VALUE,VALUEUOM,STORETIME,CGID,STOPPED,NEWBOTTLE,ISERROR
0,344,21219,177991.0,225765.0,2142-09-08 10:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
1,345,21219,177991.0,225765.0,2142-09-08 12:00:00,40055,200.0,ml,2142-09-08 12:08:00,17269,NaN,NaN,NaN
2,346,21219,177991.0,225765.0,2142-09-08 13:00:00,40055,120.0,ml,2142-09-08 13:39:00,17269,NaN,NaN,NaN
3,347,21219,177991.0,225765.0,2142-09-08 14:00:00,40055,100.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN
4,348,21219,177991.0,225765.0,2142-09-08 16:00:00,40055,200.0,ml,2142-09-08 16:17:00,17269,NaN,NaN,NaN


In [189]:
output_events.shape

(4349218, 13)

* `OUTPUTEVENTS` table contains all measurements related to output for a given patient
* Nothing interesting for this project.

### 24. `PATIENTS.csv.gz`

In [190]:
patients = pd.read_csv('PATIENTS.csv.gz', compression = 'gzip')
patients.head()

,ROW_ID,SUBJECT_ID,GENDER,DOB,DOD,DOD_HOSP,DOD_SSN,EXPIRE_FLAG
0,234,249,F,2075-03-13 00:00:00,NaN,NaN,NaN,0
1,235,250,F,2164-12-27 00:00:00,2188-11-22 00:00:00,2188-11-22 00:00:00,NaN,1
2,236,251,M,2090-03-15 00:00:00,NaN,NaN,NaN,0
3,237,252,M,2078-03-06 00:00:00,NaN,NaN,NaN,0
4,238,253,F,2089-11-26 00:00:00,NaN,NaN,NaN,0


In [191]:
patients.shape

(46520, 8)

* `PATIENTS` is mainly about the `gender`, `DOB`, `DOD` of the patient.
* `gender`, `DOB` are required, the latter along with `ADMITIME` could calculate the patient age.

### 25. `PRESCRIPTIONS.csv.gz`

In [192]:
prescriptions = pd.read_csv('PRESCRIPTIONS.csv.gz', compression = 'gzip')
prescriptions.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTDATE,ENDDATE,DRUG_TYPE,DRUG,DRUG_NAME_POE,DRUG_NAME_GENERIC,FORMULARY_DRUG_CD,GSN,NDC,PROD_STRENGTH,DOSE_VAL_RX,DOSE_UNIT_RX,FORM_VAL_DISP,FORM_UNIT_DISP,ROUTE
0,2214776,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Tacrolimus,Tacrolimus,Tacrolimus,TACR1,021796,469061711.0,1mg Capsule,2,mg,2,CAP,PO
1,2214775,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Warfarin,Warfarin,Warfarin,WARF5,006562,56017275.0,5mg Tablet,5,mg,1,TAB,PO
2,2215524,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Heparin Sodium,NaN,NaN,HEPAPREMIX,006522,338055002.0,"25,000 unit Premix Bag","25,000",UNIT,1,BAG,IV
3,2216265,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,BASE,D5W,NaN,NaN,HEPBASE,NaN,0.0,HEPARIN BASE,250,ml,250,ml,IV
4,2214773,6,107064,NaN,2175-06-11 00:00:00,2175-06-12 00:00:00,MAIN,Furosemide,Furosemide,Furosemide,FURO20,008208,54829725.0,20mg Tablet,20,mg,1,TAB,PO


In [193]:
prescriptions.shape

(4156450, 19)

* `PRESCRIPTIONS` is about the prescription records for the corresponding patient.
* Nothing interesting for this project

### 26. `PROCEDUREEVENTS_MV.csv.gz`

In [194]:
procedure_events_mv = pd.read_csv('PROCEDUREEVENTS_MV.csv.gz', compression = 'gzip')
procedure_events_mv.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,STARTTIME,ENDTIME,ITEMID,VALUE,VALUEUOM,LOCATION,LOCATIONCATEGORY,STORETIME,CGID,ORDERID,LINKORDERID,ORDERCATEGORYNAME,SECONDARYORDERCATEGORYNAME,ORDERCATEGORYDESCRIPTION,ISOPENBAG,CONTINUEINNEXTDEPT,CANCELREASON,STATUSDESCRIPTION,COMMENTS_EDITEDBY,COMMENTS_CANCELEDBY,COMMENTS_DATE
0,379,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225401,1.0,None,NaN,NaN,2145-03-12 23:04:00,20855,5092605,5092605,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
1,380,29070,115071,232563.0,2145-03-12 23:04:00,2145-03-12 23:05:00,225454,1.0,None,NaN,NaN,2145-03-12 23:04:00,20855,2538599,2538599,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
2,381,29070,115071,232563.0,2145-03-12 23:05:00,2145-03-18 20:01:00,225792,8456.0,hour,NaN,NaN,2145-03-18 20:01:39,20889,2383835,2383835,Ventilation,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN
3,382,29070,115071,232563.0,2145-03-12 23:36:00,2145-03-12 23:37:00,225402,1.0,None,NaN,NaN,2145-03-12 23:36:00,20855,9355062,9355062,Procedures,NaN,Electrolytes,0,0,0,FinishedRunning,NaN,NaN,NaN
4,383,29070,115071,232563.0,2145-03-13 01:27:00,2145-03-16 16:00:00,224560,5193.0,min,Right IJ,Invasive Venous,2145-03-16 17:57:00,20855,466169,466169,Invasive Lines,NaN,Task,1,0,0,FinishedRunning,NaN,NaN,NaN


In [196]:
procedure_events_mv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258066 entries, 0 to 258065
Data columns (total 25 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ROW_ID                      258066 non-null  int64  
 1   SUBJECT_ID                  258066 non-null  int64  
 2   HADM_ID                     258066 non-null  int64  
 3   ICUSTAY_ID                  257978 non-null  float64
 4   STARTTIME                   258066 non-null  object 
 5   ENDTIME                     258066 non-null  object 
 6   ITEMID                      258066 non-null  int64  
 7   VALUE                       258066 non-null  float64
 8   VALUEUOM                    258066 non-null  object 
 9   LOCATION                    52612 non-null   object 
 10  LOCATIONCATEGORY            52612 non-null   object 
 11  STORETIME                   258066 non-null  object 
 12  CGID                        258066 non-null  int64  
 13  ORDERID       

* `PROCEDUREEVENTS_MV` is about the record of medical procedure (including emergency & non-emergency) performed on each patient.
* It could be useful through binary labelling after joining the table by `HADM_ID` or `SUBJECT_ID`

### 27. `PROCEDURES_ICD.csv.gz`

In [197]:
procedures_icd = pd.read_csv('PROCEDURES_ICD.csv.gz', compression = 'gzip')
procedures_icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,944,62641,154460,3,3404
1,945,2592,130856,1,9671
2,946,2592,130856,2,3893
3,947,55357,119355,1,9672
4,948,55357,119355,2,331


In [198]:
procedures_icd.shape

(240095, 5)

* `PROCEDURES_ICD` stores the ICD9 code of the procedure conducted on each patient.
* <font color = 'orange'>Potentially useful</font> on identifying exact the procedure that patients underwent by joining through IDs.

### 28. `README.md`

In [101]:
readme = open('README.md', 'r')
print(readme.read())
readme.close()

# This is the default folder for files exported by the Makefile



I thought there was a bug downloading the content. However, it does provide nothing...

### 29. `SERVICES.csv.gz`

In [199]:
service = pd.read_csv('SERVICES.csv.gz', compression = 'gzip')
service.head()

,ROW_ID,SUBJECT_ID,HADM_ID,TRANSFERTIME,PREV_SERVICE,CURR_SERVICE
0,758,471,135879,2122-07-22 14:07:27,TSURG,MED
1,759,471,135879,2122-07-26 18:31:49,MED,TSURG
2,760,472,173064,2172-09-28 19:22:15,NaN,CMED
3,761,473,129194,2201-01-09 20:16:45,NaN,NB
4,762,474,194246,2181-03-23 08:24:41,NaN,NB


In [200]:
service.shape

(73343, 6)

* `SERVICES` is a spreadsheet storing the abstracted transferring record (service change) for each patient.
* <font color = 'orange'>If this project also requires predicting transferring, it would be useful by joining IDs.</font>

### 30. `SHA256SUMS.txt`

In [108]:
sha256sums = open('SHA256SUMS.txt', 'r')
print(sha256sums.read())
sha256sums.seek(0)
print(f'This document has {len(sha256sums.readlines())} lines.')
sha256sums.close()

d035bca700423dba4ea74f13389362596941c09914255008f89d1f656dbebbca D_LABITEMS.csv.gz
6a1107b6a6f11ab37ac62b3ef3901db2c5f973c5794951141efd0e268e4a9c9e PROCEDUREEVENTS_MV.csv.gz
f0800ce01fede963f2f2250b87fd983bbeb08387649c854fe76157a510b7eb67 CAREGIVERS.csv.gz
3edbc997c8b82ec6820fd478cc5588569a376373c64aad143e61757b594dc9ec DRGCODES.csv.gz
1f52534074bf0840bb002aa8fa16da1933d01b8cf2cc92f3e30ec42e04685b40 INPUTEVENTS_MV.csv.gz
4f19505634ba11b369e8855da515569628ddb6b41025b797594f697c1c9264bd INPUTEVENTS_CV.csv.gz
b15c7ee000b53fbd4e75a19a23a9a99c6969613bcd33cab8beaaea1ab14ec140 D_CPT.csv.gz
79d86315dde6e1121f7a88f62c2b91b7b835364ac1e777248a58e4f180224ae6 LABEVENTS.csv.gz
16414f0a05b281d34e8e5cfae9997a9eadd1bd22d80914a29746ef7d0597bd94 checksum_md5_zipped.txt
e8a66c2ad7433454c0afeb31ee1b952d16bff6f68e278b2e89ef640103f5ba60 README.md
5961ac36d8bfbd399088811f9a1d30bd6f482b23a84628593d3122b5a6be8bc4 CHARTEVENTS.csv.gz
457f7d96859490874486ef0ce4ece10985082cbccada680d668650ddca73a0e1 CALLOUT.csv.gz


This is a namelist of all the file inside the dataset but this file itself, including `.csv` and non `.csv`.

### 31. `TRANSFERS.csv.gz`

In [201]:
transfers = pd.read_csv('TRANSFERS.csv.gz', compression = 'gzip')
transfers.head()

,ROW_ID,SUBJECT_ID,HADM_ID,ICUSTAY_ID,DBSOURCE,EVENTTYPE,PREV_CAREUNIT,CURR_CAREUNIT,PREV_WARDID,CURR_WARDID,INTIME,OUTTIME,LOS
0,657,111,192123,254245.0,carevue,transfer,CCU,MICU,7.0,23.0,2142-04-29 15:27:11,2142-05-04 20:38:33,125.19
1,658,111,192123,NaN,carevue,transfer,MICU,NaN,23.0,45.0,2142-05-04 20:38:33,2142-05-05 11:46:32,15.13
2,659,111,192123,NaN,carevue,discharge,NaN,NaN,45.0,NaN,2142-05-05 11:46:32,NaN,NaN
3,660,111,155897,249202.0,metavision,admit,NaN,MICU,NaN,52.0,2144-07-01 04:13:59,2144-07-01 05:19:39,1.09
4,661,111,155897,NaN,metavision,transfer,MICU,NaN,52.0,32.0,2144-07-01 05:19:39,2144-07-01 06:28:29,1.15


In [205]:
transfers.shape

(261897, 13)

* `TRANSFERS` is a spreadsheet storing the admission, discharge, transferring event for each patient.
* <font color = 'orange'>If we have to predict patient transferration, we would have to join the table by IDs, before fitering `EVENTTYPE` = 'transfer'</font>

### Therefore, the data potentially useful to predict patient triage / the need of emergency procedure are as follows:


<br>
However, Apart from that, <font color = 'red'><b>there's no such information about the patient triage level.</b></font> Therefore, I am going to predict the need of emergency medical intervention of the patient triage, which means I need the <font color = 'red'><b>definition & clarification of the patient with "Requirement of Emergency Procedure".</b></font>